In [392]:
import cv2 as cv
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import random
from itertools import combinations
import anvil.server
import anvil.media

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
        
colour_model = tf.keras.models.load_model("models/colour")
shape_model =  tf.keras.models.load_model("models/shape")
fill_model = tf.keras.models.load_model("models/fill")
number_model =  tf.keras.models.load_model("models/number")

vocab_colour = sorted(["red", "green", "blue"])
vocab_shape = sorted(["oval", "diamond", "squiggle"])
vocab_number = sorted([1,2,3])
vocab_fill = sorted(["filled", "open", "shaded"])
    
def show(imgs):
    for im in imgs:
        #im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
        im = Image.fromarray(im)
        im.thumbnail((400,400))
        display(im)
        

In [382]:
# https://anvil.works/build#app:SH7F4TDWQQUSHC7F

anvil.server.connect("RWL6DCXP5PA5JSHIYP55ZV34-SH7F4TDWQQUSHC7F")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [321]:
sample_imgs = [cv.cvtColor(cv.imread(x), cv.COLOR_BGR2RGB) for x in [f"photos/{f}" for f in os.listdir("photos")]]


In [331]:
# Utilities

def resize(img, size):
    scale = size / max(img.shape)
    
    # cv.resize seems to need a destination image, and also returns a value. Weird.
    x = img.copy()
    x = cv.resize(x, (0,0), x, scale, scale, cv.INTER_AREA)
    return x, scale

def threshold(img):
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    
    # https://docs.opencv.org/master/d7/d4d/tutorial_py_thresholding.html
    x = cv.adaptiveThreshold(gray,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, int(gray.shape[1]/20)*2+1, 2)
    
    kernel = np.ones((3,3),np.uint8)
    x = cv.erode(x, kernel, iterations=1)
    
    return x

def contours(img):
    # Nice contour docs: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html
    # Find all contours, sorted by size
    contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_TC89_KCOS)
    contours.sort(key=cv.contourArea,reverse=True)
    
    # Simplify contours
    simple_contours = [cv.approxPolyDP(c, 0.05*cv.arcLength(c, True),True) for c in contours]
    
    # Keep contours whose simple shape:
    # - roughly matches the original shape, 
    # - is a rectangle, 
    # - has area > 500.
    
    simple_contours = [s for s,c in zip(simple_contours, contours) if cv.matchShapes(c,s,cv.CONTOURS_MATCH_I1,0) < 0.2 and len(s) == 4 and cv.contourArea(s) > 500]
                
    return simple_contours
    
def extract_contours(img, contours, w=300, h=None):
    if h is None:
        h = int(2 * (w / 3.0))
    cards = []
    target = np.float32([[0,0], [0,h], [w,h], [w,0]])
    for s in contours:
        # Roll contour until we're at the start of a short side.
        s = np.roll(s, 2 * np.argmax([cv.arcLength(s[i:i+2], False) for i in range(3)]))
        
        s = np.float32(np.roll(s,2))
        M = cv.getPerspectiveTransform(s, target)
        persp = cv.warpPerspective(img, M, (w,h))
        cards.append(persp)
        
    return cards

    
# Takes a cv Image and returns a list of card images

def image_to_cards(img, verbose=True):
    
    # Resize image to ~400px for thresholding
    small, scale = resize(img, 400)
    
    # Threshold image
    bw = threshold(small)
    
    # Find contours and upscale back to original image size
    card_contours = [np.int32(c / scale) for c in contours(bw)]
    outlined = img.copy()
    cv.drawContours(outlined, card_contours, -1, (255,255,0), 20)
    
    # Extract cards
    cards = extract_contours(img, card_contours, 120)
    
    return bw, outlined, card_contours, cards
    

In [395]:
def analyse_image(img):
    bw, outlined, contours, cards = image_to_cards(img)
    show([bw, outlined])
    
    cards_array = np.uint8(cards)

    colour_scores = tf.nn.softmax(colour_model(cards_array))
    shape_scores = tf.nn.softmax(shape_model(cards_array))
    fill_scores = tf.nn.softmax(fill_model(cards_array))
    number_scores = tf.nn.softmax(number_model(cards_array))
    
    colours = [vocab_colour[i] for i in tf.argmax(colour_scores, 1)]
    shapes = [vocab_shape[i] for i in tf.argmax(shape_scores, 1)]
    fills = [vocab_fill[i] for i in tf.argmax(fill_scores, 1)]
    numbers = [vocab_number[i] for i in tf.argmax(number_scores, 1)]
    
    results = zip(
        cards, contours,
        colours, shapes, fills, numbers, 
        np.max(colour_scores,1), np.max(shape_scores,1), np.max(fill_scores,1), np.max(number_scores,1)
    )
    
    return results
    
def print_results(results):
    for card, contour, colour, shape, fill, number, colour_score, shape_score, fill_score, number_score in results:
        show([card])
        print(f"{number} ({number_score:.1%})")
        print(f"{fill} ({fill_score:.1%})")
        print(f"{colour} ({colour_score:.1%})")
        print(f"{shape} ({shape_score:.1%})")
        print("----------------")
        
def highlight_set(img, card_set, set_index):
    colors = [(255,255,0), (255,0,255), (0,255,255)]
    w = img.shape[0]*0.02
    print(img.shape)
    cv.drawContours(img, [c[1] for c in card_set], -1, colors[set_index % len(colors)], int(w - set_index*0.45*w))
    
    

In [397]:


def find_sets_in_img(img):
    results = analyse_image(img)
    #print_results(results)

    result_img = img.copy()
    set_idx = 0
    for cards in combinations(results, 3):

        colours = [c[2] for c in cards]
        shapes = [c[3] for c in cards]
        fills = [c[4] for c in cards]
        numbers = [c[5] for c in cards]

        features = [colours, shapes, fills, numbers]
        valid_set = True
        for f in features:
            valid_set = valid_set and (len(set(f)) == 3 or len(set(f)) == 1)

        if valid_set:
            highlight_set(result_img, cards, set_idx)
            set_idx += 1
            print("--- SET ---")
            print(colours, shapes, fills, numbers)
            show([c[0] for c in cards])
            
    return result_img if set_idx > 0 else None

@anvil.server.callable
def find_sets(img_media):
    img_data = img_media.get_bytes()
    img = cv.cvtColor(cv.imdecode(np.frombuffer(img_data, np.uint8), cv.IMREAD_COLOR), cv.COLOR_BGR2RGB)
    
    result = find_sets_in_img(img)
    
    show([result])
    
    with anvil.media.TempFile() as file_name:
        file_name += ".jpg"
        result_bgr = cv.cvtColor(result, cv.COLOR_RGB2BGR)
        cv.imwrite(file_name,result_bgr)
    
        return anvil.media.from_file(file_name, "image/jpeg")

#result_img = find_sets_in_img(sample_imgs[15])
#show([result_img])
    


Anvil websocket closed (code 1006, reason=Going away)
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Reconnection failed. Waiting 10 seconds, then retrying.
